In [ ]:
import cv2
import numpy as np

roi = None
drag_start = None
mouse_status = 0
tracking_start = False
def onMouse(event,x,y,flags,param = None):
    global roi
    global drag_start
    global mouse_status
    global tracking_start
    if event == cv2.EVENT_LBUTTONDOWN:
        drag_start = (x,y)
        mouse_status = 1
        tracking_start = False
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags == cv2.EVENT_FLAG_LBUTTON:
            xmin = min(x,drag_start[0])
            ymin = min(y,drag_start[1])
            xmax = max(x,drag_start[0])
            ymax = min(y,drag_start[1])
            roi = (xmin,ymin,xmax,ymax)
            mouse_status = 2
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_status = 3
        
        
cv2.namedWindow('tracking')
cv2.setMouseCallback('tracking',onMouse)

cap = cv2.VideoCapture('Aircraft.avi')
if(not cap.isOpened()):
    print('Error opening video')
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                     int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
roi_mask = np.zeros((height,width))
term_crit = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS,10,1)
    
q = 1e-5 #프로세스 잡음 공분산Q
r = 0.01 #측정잡음 공분산R
dt = 1 #상태변환행렬에 들어갈 시간변화 단위(1초마다 추정값 변경)
KF = cv2.KalmanFilter(4,2,0) #상태변환행렬과 측정행렬의 차원을 설정
KF.transitionMatrix = np.array([[1,0,dt,0], #상태변환행렬 A
                                [0,1,0,dt],
                                [0,0,1,0],
                                [0,0,0,1]], np.float32)
KF.measurementMatrix = np.array([[1,0,0,0], #측정행렬 H
                                [0,1,0,0]],np.float32)
    
t=0
while True:
    ret,frame = cap.read()
    if not ret : break
    t += 1
    print('t=',t)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv,(0,0,200),(100,250,255))

    if mouse_status ==2:
        x1,y1,x2,y2 = roi
        cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,0,),2)

    if mouse_status ==3:
        print('initialize...')
        mouse_status = 0
        x1,y1,x2,y2 = roi
        mask_roi = mask[y1:y2, x1:x2]
        hsv_roi = hsv[y1:y2, x1:x2]

        hist_roi = cv2.calcHist([hsv_roi],[0],mask_roi,[16],[0,180])
        cv2.normalize(hist_roi, hist_roi, 0,255,cv2.NORM_MINMAX)
        track_window = (x1, y1, x2-x1,y2-y1)

        KF.processNoiseCov = q * np.eye(4, dtype = np.float32)
        KF.measurementNoiseCov = r * np.eye(2, dtype = np.float32)
        KF.errorCovPost = np.eye(4, dtype = np.float32)

        cx = (x1+x2)/2
        cy = (y1+y2)/2
        KF.statePost = np.array([[cx],[cy],[0.],[0.]],dtype = np.float32)
        tracking_start = True

        if tracking_start:
            predict = KF.predict() #예측단계 연산
        backP = cv2.calcBackProject([hsv],[0], hist_roi,[0,180],1)
        backP &=mask
        cv2.imshow('backP', backP)

        ret, track_window = cv2.meanShift(backP, track_window1,term_crit)
        x,y,w,h = track_window
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.circle(frame,(round(x),round(y)),5,(0,0,255),-1)     
            
        z = np.array([[x],[y]],dtype = np.float32) #추정단계 연산
        estimate = KF.correct(z)
        estimate = np.int0(estimate)

            
        x2, y2 = estimate[0][0], estimate[1][0]
        cv2.rectangle(frame,(x2,y2),(x2+w,y2+h),(0,0,0),2)
        cv2.circle(frame,(cx2,cy2),5,(255,0,0),-1)
        track_window = x2,y2,w,h


cv2.imshow('tracking',frame)
key = cv2.waitKey(25)
if key == 27:
    break
if cap.isOpened():
    cap.release();
cv2.destroyAllWindows()
